In [1]:
images = sqlContext.read.parquet("file:/Users/vk/data/ML_MP_JR/parquet/images/*.parquet")

In [2]:
images.printSchema()

root
 |-- image: vector (nullable = true)
 |-- label: long (nullable = true)



In [3]:
images.show()

+--------------------+-----+
|               image|label|
+--------------------+-----+
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,0.489999...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,0.699999...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,0.699999...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
|[1.0,1.0,1.0,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [5]:
from pyspark.sql.functions import rand 
random_images = images.orderBy(rand())

In [6]:
from densenet import DenseNet

Using TensorFlow backend.


In [7]:
model = DenseNet(nb_classes = 3, img_dim = (500, 314, 3), depth = 10, 
                 nb_dense_block = 2, growth_rate = 3, 
                 dropout_rate= 0.5, nb_filter = 12)

tf
tf


In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500, 314, 3)  0                                            
__________________________________________________________________________________________________
initial_conv2D (Conv2D)         (None, 12, 314, 3)   54000       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 12, 314, 3)   48          initial_conv2D[0][0]             
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 12, 314, 3)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (

In [9]:
images_labels = random_images

In [10]:
from distkeras.trainers import *
from distkeras.predictors import *
from distkeras.transformers import *
from distkeras.evaluators import *
from distkeras.utils import *

In [11]:
transformer = ReshapeTransformer("image", "image_matrix", (500, 314, 3))

In [12]:
new_images = transformer.transform(images)

In [13]:
new_images.printSchema()

root
 |-- image: vector (nullable = true)
 |-- label: long (nullable = true)
 |-- image_matrix: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: double (containsNull = true)



In [14]:
(training_set, test_set) = new_images.randomSplit([0.6, 0.4])

In [15]:
optimizer_mlp = 'adam'
loss_mlp = 'categorical_crossentropy'

In [18]:
trainer = DOWNPOUR(keras_model=model, worker_optimizer=optimizer_mlp, loss=loss_mlp, 
                   num_workers=1, batch_size=4, communication_window=5, num_epoch=1,
                   features_col="image_matrix", label_col = "label")
trainer.set_parallelism_factor(1)

In [ ]:
trained_model = trainer.train(training_set)

In [ ]:
trainer.get_training_time()

In [ ]:
trainer.parameter_server.num_updates

In [23]:
trained_model.weights

[<tf.Variable 'initial_conv2D_1/kernel:0' shape=(3, 3, 3, 12) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1_1/gamma:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1_1/beta:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'conv2d_1_1/kernel:0' shape=(3, 3, 12, 3) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_2_1/gamma:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_2_1/beta:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'conv2d_2_1/kernel:0' shape=(3, 3, 15, 3) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_3_1/gamma:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_3_1/beta:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'conv2d_3_1/kernel:0' shape=(1, 1, 18, 18) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_4_1/gamma:0' shape=(250,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_4_1/beta:0' shape=(250,) dtype=float32_ref>,
 <tf.Variable 'conv2d_4_1/kernel:0' shape=(3,